In [ ]:
from functools import partial

import torch.distributions
import transformer_lens
from transformer_lens import HookedTransformer
from IPython.display import SVG

import core
from utils import sankey_diagram_of_connectome

%load_ext autoreload
%autoreload 2

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")

In [ ]:
model = HookedTransformer.from_pretrained("attn-only-4l")

In [ ]:
threshold = 0.3
prompt = "When Mary and John went to the store, John gave a book to"
corrupt_prompt = "When Tom and Sarah went to the store, Felix gave a book to"
c = core.connectom(model, prompt,
                   core.logit_diff_metric(model, ' Mary', ' John'),
                   core.ZeroPattern(),
                   # core.CorruptIntervention(model, prompt, corrupt_prompt),
                   # core.CropIntervention(model, prompt),
                   # core.BasicStrategy(),
                   # d.BacktrackBisectStrategy(threshold),
                   # d.BacktrackingStrategy(threshold),
                   # core.BisectStrategy(threshold),
                   core.SplitStrategy(model, prompt, threshold),
                   )
core.plot_attn_connectome(model, prompt, c).show()
# graph = core.plot_graphviz_connectome(model, prompt, c, threshold=threshold).pipe('svg').decode('utf-8')
# SVG(graph)


In [ ]:
graph = core.plot_graphviz_connectome(model, prompt, c, threshold=0.2).pipe('svg').decode('utf-8')
SVG(graph)

In [ ]:
model = HookedTransformer.from_pretrained("attn-only-4l")
threshold = 0.3
prompt = '''def port(self, load, size, file, last):
    """oil column piece

    :param load: crime population
    :param size: unit dark
    :param'''
corrupt_prompt = prompt.replace("load", "banana").replace("size", "apple").replace("file", "pear").replace("last", "orange")

c = core.connectom(model, prompt,
                   core.logit_diff_metric(model, ' file', ' self', ' load', ' size', ' last'),
                   core.ZeroPattern(),
                   # core.CorruptIntervention(model, prompt, corrupt_prompt),

                   # core.BasicStrategy(),
                   core.SplitStrategy(model, prompt, threshold),
                   # core.BacktrackBisectStrategy(threshold),
                   # d.BacktrackingStrategy(threshold),
                   # core.BisectStrategy(threshold),
                   )

In [ ]:
graph_threshold = 0.4
core.plot_attn_connectome(model, prompt, c).show()
graph = core.plot_graphviz_connectome(model, prompt, c, graph_threshold).pipe('svg').decode('utf-8')
SVG(graph)


In [ ]:
transformer_lens.utils.test_prompt(prompt, ' file', model)
# logit, best = model(prompt)[0, -1].topk(10)
# for b, l in zip(model.to_str_tokens(best), logit):
#     print(b, l.item())
print(prompt)

In [ ]:
graph = core.plot_graphviz_connectome(model, prompt, c, threshold=0.4).pipe('svg').decode('utf-8')
SVG(graph)


In [ ]:
torch.topk()
dampening = 0.25
c = core.connectom(model, prompt,
                   core.logit_diff_metric(model, ' Mary', ' John'),
                   # d.CorruptIntervention(model, prompt, corrupt_prompt),
                   core.DampenIntervention(dampening),
                   # d.ZeroPattern(),
                   core.BasicStrategy(),
                   )
core.plot_attn_connectome(model, prompt, c).show()
graph = core.plot_graphviz_connectome(model, prompt, c, threshold=0.05).pipe('svg').decode('utf-8')
SVG(graph)

In [ ]:
# Finding the log-probs on the prompt tokens
prompt = "When Mary and John went to the store, John gave a book to Mary."
log_probs = model(prompt)[0].log_softmax(-1)  # (seq_len, vocab_size)
tokens = model.to_tokens(prompt)[0]
tokens_str = model.to_str_tokens(tokens)
print(tokens.shape)
print(log_probs.shape)
correct_logprobs = log_probs[torch.arange(len(tokens) - 1), tokens[1:]]
print(correct_logprobs.shape)
for i, (t, n, l) in enumerate(zip(tokens_str, tokens_str[1:], correct_logprobs)):
    print(f"{i:2d} {t!r} {l.item():.2f} -> {n!r}")

import plotly.express as px
px.line(x=[f'{i} {t!r} -> {n!r}' for i, (t, n) in enumerate(zip(tokens_str, tokens_str[1:]))],
        y=correct_logprobs.detach() * 0)

In [ ]:
prompt = '''def port(self, load, size, file, last):
    """oil column piece

    :param load: crime population
    :param size: unit dark
    :param'''

print(prompt)
s = core.SplitStrategy(model, prompt, 0.1,
                       (
                           '\n\n',
                           tuple('.!?'),
                           tuple(',:;'),
                       )
                       )
s.show_tree()